In [1]:
import os
import chardet
import os
import re
import pandas as pd

In [2]:

# Set the folder path
folder_path = "/Users/thyag/Downloads/intershala assignment/StopWords"

# Initialize an empty list to store all words
all_words = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a text file
    if filename.endswith(".txt"):
        # Open the file in binary mode
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "rb") as file:
            raw_data = file.read()
            # Detect the encoding
            encoding = chardet.detect(raw_data)["encoding"]

        # Open the file in text mode with the detected encoding
        with open(file_path, "r", encoding=encoding) as file:
            # Read the contents of the file
            contents = file.read()
            # Split the contents into words
            words = contents.split()
            # Add the words to the all_words list
            all_words.extend(words)

stopwords= [word.lower() for word in all_words]

In [3]:
# Load the positive and negative word lists
positive_word_list_path = "/Users/thyag/Downloads/intershala assignment/MasterDictionary/positive-words.txt"
negative_word_list_path = "/Users/thyag/Downloads/intershala assignment/MasterDictionary/negative-words.txt"

positive_words = set()
negative_words = set()

# Detect encoding for positive word list
with open(positive_word_list_path, "rb") as file:
    raw_data = file.read()
    encoding = chardet.detect(raw_data)["encoding"]

with open(positive_word_list_path, "r", encoding=encoding) as file:
    for line in file:
        word = line.strip().lower()
        positive_words.add(word)

# Detect encoding for negative word list
with open(negative_word_list_path, "rb") as file:
    raw_data = file.read()
    encoding = chardet.detect(raw_data)["encoding"]

with open(negative_word_list_path, "r", encoding=encoding) as file:
    for line in file:
        word = line.strip().lower()
        negative_words.add(word)

In [4]:
df = pd.DataFrame()

In [5]:
# Directory containing the files
directory = "/Users/thyag/Downloads/intershala assignment/testFiles"

# Initialize an empty list to store file paths
file_paths = []

# Loop over files in the directory
for filename in os.listdir(directory):
    # Check if the file is a text file
    if filename.endswith(".txt"):
        # Full path to the file
        file_path = os.path.join(directory, filename)
        
        # Append the file path to the list
        file_paths.append(file_path)

In [6]:
for i in range(100):
    text_file_path = file_paths[i]
    with open(text_file_path, "r", encoding="utf-8") as file:
        # Skip the first line
        next(file)
        text = file.read()

    # Convert the text to lowercase
    text = text.lower()

    # Split the text into words
    text_words = text.split()
    # Remove the stop words from the text words
    filtered_words = [word for word in text_words if word not in stopwords]

    new_filtered_words = ' '.join(filtered_words)

    # Calculate positive and negative word scores
    positive_score = 0
    negative_score = 0

    for word in filtered_words:
        if word in positive_words:
            positive_score += 1
        elif word in negative_words:
            negative_score -= 1

    polarity_score = (positive_score - abs(negative_score)) / (positive_score + abs(negative_score) + 0.000001)
    Subjectivity_Score = (positive_score + abs(negative_score))/ ((len(filtered_words)) + 0.000001)


    def average_sentence_length(text):
        # Count the number of sentences
        sentences = text.split('.')
        num_sentences = len(sentences)

        # Remove empty strings
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

        # Count the number of words
        num_words = sum(len(sentence.split()) for sentence in sentences)

        # Calculate the average sentence length
        if num_sentences > 0:
            average_length = num_words / num_sentences
        else:
            average_length = 0

        return average_length


    def count_syllables(word):
        """Count the number of syllables in a word."""
        vowels = "aeiouy"
        num_syllables = 0
        prev_char_was_vowel = False
        word = word.lower()
        
        # Handle special cases
        if word.endswith("es") or word.endswith("ed"):
            word = word[:-2]

        for char in word:
            if char in vowels:
                if not prev_char_was_vowel:
                    num_syllables += 1
                prev_char_was_vowel = True
            else:
                prev_char_was_vowel = False

        # Adjust for words ending with "le"
        if word.endswith("le") and len(word) > 2:
            num_syllables += 1

        # Ensure at least one syllable counted for non-empty words
        if len(word) > 0 and num_syllables == 0:
            num_syllables = 1

        return num_syllables

    def percentage_complex_words(text):
        # Tokenize the text into words
        words = text.split()

        # Count the number of complex words
        num_complex_words = sum(1 for word in words if count_syllables(word) > 2)

        # Calculate the total number of words
        num_words = len(words)

        # Calculate the percentage of complex words
        if num_words > 0:
            percentage = (num_complex_words / num_words) * 100
        else:
            percentage = 0

        return percentage

    # Example usage:
    #text = "This is a sample sentence with some complex words like 'complicated' and 'entanglement'."

    def count_syllables(word):
        word = word.lower()
        vowels = "aeiouy"
        count = 0
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith("e"):
            count -= 1
        if count == 0:
            count += 1
        return count

    def is_complex_word(word):
        return count_syllables(word) >= 3

    def gunning_fog_index(text):
        # Split text into sentences
        sentences = re.split(r'[.!?]', text)
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        
        # Split text into words
        words = re.findall(r'\w+', text)
        
        # Count complex words
        complex_words = [word for word in words if is_complex_word(word)]
        
        total_words = len(words)
        total_sentences = len(sentences)
        total_complex_words = len(complex_words)
        
        if total_sentences == 0:  # Avoid division by zero
            return 0
        
        gfi = 0.4 * ((total_words / total_sentences) + 100 * (total_complex_words / total_words))
        
        return gfi

    gfi_score = gunning_fog_index(new_filtered_words)



    def average_words_per_sentence(text):
        # Define a simple sentence tokenizer based on punctuation
        sentences = re.split(r'[.!?]+', text)
        
        # Remove any empty sentences that may result from the split
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        
        total_sentences = len(sentences)
        total_words = sum(len(sentence.split()) for sentence in sentences)
        
        # Calculate the average number of words per sentence
        if total_sentences == 0:
            return 0
        
        average = total_words / total_sentences
        return average

    average = average_words_per_sentence(text)


    def count_syllables(word):
        """Count the number of syllables in a word."""
        vowels = "aeiouy"
        num_syllables = 0
        prev_char_was_vowel = False
        word = word.lower()
        
        # Handle special cases
        if word.endswith("es") or word.endswith("ed"):
            word = word[:-2]

        for char in word:
            if char in vowels:
                if not prev_char_was_vowel:
                    num_syllables += 1
                prev_char_was_vowel = True
            else:
                prev_char_was_vowel = False

        # Adjust for words ending with "le"
        if word.endswith("le") and len(word) > 2:
            num_syllables += 1

        # Ensure at least one syllable counted for non-empty words
        if len(word) > 0 and num_syllables == 0:
            num_syllables = 1

        return num_syllables

    def complex_word_count(text):
        # Tokenize the text into words
        words = text.split()

        # Count the number of complex words
        num_complex_words = sum(1 for word in words if count_syllables(word) > 2)

        return num_complex_words

    # Example usage:
    #text = "This is a sample sentence with some complex words like 'complicated' and 'entanglement'."

    def count_syllables(word):
        """Count the number of syllables in a word."""
        vowels = "aeiouy"
        num_syllables = 0
        prev_char_was_vowel = False
        word = word.lower()
        
        # Handle special cases
        if word.endswith("es") or word.endswith("ed"):
            word = word[:-2]

        for char in word:
            if char in vowels:
                if not prev_char_was_vowel:
                    num_syllables += 1
                prev_char_was_vowel = True
            else:
                prev_char_was_vowel = False

        # Adjust for words ending with "le"
        if word.endswith("le") and len(word) > 2:
            num_syllables += 1

        # Ensure at least one syllable counted for non-empty words
        if len(word) > 0 and num_syllables == 0:
            num_syllables = 1

        return num_syllables

    def syllable_count_per_word(text):
        # Tokenize the text into words
        words = text.split()

        # Count the number of syllables in each word
        syllable_counts = [count_syllables(word) for word in words]

        return syllable_counts

    def average_syllables_per_word(text):
        # Get the counts of syllables per word
        syllable_counts = syllable_count_per_word(text)
        
        # Calculate the average syllable per word
        total_syllables = sum(syllable_counts)
        total_words = len(syllable_counts)
        
        if total_words > 0:
            average_syllables = total_syllables / total_words
        else:
            average_syllables = 0
        
        return average_syllables

    # Example usage:
    #text = "This is a sample sentence with some complex words like 'complicated' and 'entanglement'."

    def count_personal_pronouns(text):
        # Define the list of personal pronouns
        personal_pronouns = ["I", "we", "my", "ours", "us"]
        
        # Compile regex pattern to match personal pronouns
        pattern = r"\b(?:{})\b".format("|".join(personal_pronouns))

        # Find all matches of personal pronouns in the text
        matches = re.findall(pattern, text, flags=re.IGNORECASE)

        # Exclude matches that are part of other words (e.g., "US" as a country)
        matches = [match for match in matches if match.lower() != "us"]

        # Count the occurrences of personal pronouns
        count = len(matches)

        return count

    def average_word_length(text):
        # Tokenize the text into words
        words = text.split()

        # Calculate the total number of characters in all words
        total_characters = sum(len(word) for word in words)

        # Calculate the total number of words
        total_words = len(words)

        # Calculate the average word length
        if total_words > 0:
            average_length = total_characters / total_words
        else:
            average_length = 0

        return average_length

    new_data = {
        "POSITIVE SCORE": [positive_score],
        "NEGATIVE SCORE": [negative_score],
        "POLARITY SCORE": [polarity_score],
        "SUBJECTIVITY SCORE": [Subjectivity_Score],
        "AVG SENTENCE LENGTH": [average_sentence_length(new_filtered_words)],
        "PERCENTAGE OF COMPLEX WORDS": [percentage_complex_words(new_filtered_words)],
        "FOG INDEX": [gfi_score],
        "AVG NUMBER OF WORDS PER SENTENCE": [average],
        "COMPLEX WORD COUNT": [complex_word_count(text)],
        "WORD COUNT": [len(filtered_words)],
        "SYLLABLE PER WORD": [average_syllables_per_word(text)],
        "PERSONAL PRONOUNS": [count_personal_pronouns(text)],
        "AVG WORD LENGTH": [average_word_length(text)]
    }

    new_df = pd.DataFrame(new_data)
    # Append the new DataFrame to the existing one
    df = pd.concat([df, new_df], ignore_index=True)
    df

In [7]:
df

,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,49,-5,0.814815,0.063529,8.885417,50.705882,22.126485,17.653061,454,850,1.859873,5,5.466705
1,57,-31,0.295455,0.103896,9.964706,48.996458,21.782689,20.000000,451,847,1.915294,1,5.539412
2,13,-10,0.130435,0.048729,12.200000,32.415254,15.932978,24.395349,170,472,1.678295,3,4.934109
3,19,-2,0.809524,0.075000,11.791667,45.714286,20.134452,23.375000,140,280,1.869176,1,5.439068
4,45,-28,0.232877,0.119086,8.661972,36.704731,16.757334,17.231707,254,613,1.696669,4,5.007087
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,33,-24,0.157895,0.087156,11.135593,46.024465,20.040687,19.603175,323,654,1.908279,7,5.635552
96,23,-6,0.586207,0.057426,6.405063,36.633663,15.013097,15.468354,187,505,1.670762,11,4.708436
97,27,-21,0.125000,0.087591,12.108696,48.357664,19.357823,22.653061,277,548,1.865455,7,5.520909
98,27,-21,0.125000,0.087591,12.108696,48.357664,19.357823,22.653061,277,548,1.865455,7,5.520909


In [8]:
import pandas as pd

# Load an Excel file into a DataFrame
df_index = pd.read_excel('/Users/thyag/Downloads/intershala assignment/Output Data Structure.xlsx')

df_index = df_index[['URL_ID', 'URL']]


In [9]:
df_index

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...
...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...
97,blackassign0098,https://insights.blackcoffer.com/contribution-...
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...


In [10]:
df_merged = df_index.join(df)


In [11]:
df_merged

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,49,-5,0.814815,0.063529,8.885417,50.705882,22.126485,17.653061,454,850,1.859873,5,5.466705
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,57,-31,0.295455,0.103896,9.964706,48.996458,21.782689,20.000000,451,847,1.915294,1,5.539412
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,13,-10,0.130435,0.048729,12.200000,32.415254,15.932978,24.395349,170,472,1.678295,3,4.934109
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,19,-2,0.809524,0.075000,11.791667,45.714286,20.134452,23.375000,140,280,1.869176,1,5.439068
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,45,-28,0.232877,0.119086,8.661972,36.704731,16.757334,17.231707,254,613,1.696669,4,5.007087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,33,-24,0.157895,0.087156,11.135593,46.024465,20.040687,19.603175,323,654,1.908279,7,5.635552
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,23,-6,0.586207,0.057426,6.405063,36.633663,15.013097,15.468354,187,505,1.670762,11,4.708436
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,27,-21,0.125000,0.087591,12.108696,48.357664,19.357823,22.653061,277,548,1.865455,7,5.520909
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,27,-21,0.125000,0.087591,12.108696,48.357664,19.357823,22.653061,277,548,1.865455,7,5.520909


In [12]:
df_merged.to_excel("data.xlsx", index=False)
